In [1]:
# pip install -e gym-TicTacToe/.

In [2]:
import gym_TicTacToe
import gym
import numpy as np
from IPython.display import clear_output
from agent import TicTacToeAgent
import tqdm

# initialize the tictactoe environment
env = gym.envs.make("TTT-v0", small=-1, large=20)

In [3]:
episodes = 1_000_000
agent = TicTacToeAgent(num_of_actions=env.action_space.n, 
                        gamma=0.98, 
                        lr=0.01, 
                        epsilon_min=0.1, 
                        epsilon_min_episode_reached=episodes)
agent.train()
test = False
talk_every = 10_000

ep_rewards, epl_avg = [], []
for ep in tqdm.tqdm(range(episodes)):
    state, info, done, steps = *env.reset(), False, 0
    
    agent.update_epsilon(ep)
    turn = 0
    sum_reward = 0
    
    while not done:
        action = agent.get_action(state)
        new_state, reward, done, info = env.step((action, 1))

        # if done and reward > 0:
        #     print(agent.encode_state(new_state))

        if not test:
            agent.learn(state, action, new_state, reward, done)#, talk=(done and reward>0))
        else:
            env.render()

            
        state = new_state
        sum_reward += reward
        steps += 1

    ep_rewards.append(sum_reward)
    epl_avg.append(np.mean(ep_rewards[ max(0, ep-talk_every) : ]))

    if (ep+1) % talk_every == 0:
        clear_output(wait=True)
        print("="*22, "Training resume", "="*22)
        print(f"Episode {ep+1}, average reward: { np.mean(ep_rewards[max(0, ep-talk_every):]) :.2f}")
        print(f"Perc. wins:  {(np.sum(np.array(ep_rewards[max(0, ep-talk_every):]) > 0 )/talk_every) * 100:.4}")
        print(f"Epsilon: {agent.epsilon:.4}")
        print(f"Len QTable: {len(agent.q_table)}")
        print("="*61, end='\r', flush=True)


====================== Training resume ======================
Episode 20000, average reward: -18.21
Perc. wins:  10.16
Epsilon: 0.9328
Len QTable: 3296


  1%|          | 21995/2500000 [00:29<56:07, 735.91it/s]  


KeyboardInterrupt: 

In [ ]:
# for k,v in agent.q_table.items():
#     # if (v > 0).any():
#     print(k,v)

state, info, done, steps = *env.reset(), False, 0
state, reward, done, info = env.step((1,1))
print("STATE",agent.encode_state(state))
print( agent.q_table[agent.encode_state(state)] )
print(done)

state, reward, done, info = env.step((4,1))
print(agent.encode_state(state))
print( agent.q_table[agent.encode_state(state)] )

print(done)

state, reward, done, info = env.step((7,1))
# state, reward, done, info = env.step((8,2))
print(agent.encode_state(state))
print( agent.q_table[agent.encode_state(state)] )

print(done)

STATE 010200000
[ -3.861 -10.87   -6.49  -11.23   -6.152  -6.19   -5.727  -5.785  -5.324]
False
010212000
[-9.82   -1.17   -9.56   -1.17   -1.17   -0.9795 -9.48   -9.42   -9.65  ]
False
010212010
[-10.   0. -10.   0.   0.   0. -10.   0. -10.]
True
